<a href="https://colab.research.google.com/github/Resource-Efficiency-Collective/coding-tutorials/blob/main/energy_consumption_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Energy consumption

This notebook is split into 3 tasks:


1.   US Energy Sankey example
2.   Create your own Sankey using IEA data
3.   Automate Sankey generation for any country by writing a Python function

Run the first two cells setup the notebook.





In [ ]:
# @markdown Run this cell to download data. You can view it in the panel on the left.
%%capture
"""Installation and downloads"""
# Install floweaver and display widget packages
%pip install floweaver ipysankeywidget openpyxl --upgrade

# Import necessary packages
from floweaver import *
import gdown, os
from google.colab import files

# Function to download and unzip the examples file from Google Drive -> You can view them in the left panel
def download_and_unzip_from_google_drive(file_id = '1qriY29v7eKJIs07UxAw5RlJirfwuLnyP', destination_folder = 'example_data'):
    if os.path.exists(destination_folder):
      print('zip file already downloaded. If you want to reload then please delete the example_data folder.')
      return

    else:
      download_url = f"https://drive.google.com/uc?id={file_id}"
      !wget -O temp.zip $download_url

      os.makedirs(destination_folder)
      !unzip temp.zip -d $destination_folder

      os.remove("temp.zip")

download_and_unzip_from_google_drive()

In [ ]:
"""Display setup"""
# Enable widget display for Sankeys in Colab
from google.colab import output
output.enable_custom_widget_manager()

## Task 1 - US example

Step through this section to see an example for the US based on the [Sankey diagrams of US energy consumption from the Lawrence Livermore National Laboratory](https://flowcharts.llnl.gov/).

In [ ]:
"""Load the dataset"""
dataset = Dataset.from_csv('example_data/us-energy-consumption.csv',
                           dim_process_filename='example_data/us-energy-consumption-processes.csv')

In [ ]:
"""Define the order the nodes appear in"""
sources = ['Solar', 'Nuclear', 'Hydro', 'Wind', 'Geothermal',
           'Natural_Gas', 'Coal', 'Biomass', 'Petroleum']

uses = ['Residential', 'Commercial', 'Industrial', 'Transportation']

In [ ]:
"""define the Sankey diagram definition"""
nodes = {
    'sources': ProcessGroup('type == "source"', Partition.Simple('process', sources), title='Sources'),
    'imports': ProcessGroup(['Net_Electricity_Import'], title='Net electricity imports'),
    'electricity': ProcessGroup(['Electricity_Generation'], title='Electricity Generation'),
    'uses': ProcessGroup('type == "use"', partition=Partition.Simple('process', uses)),

    'energy_services': ProcessGroup(['Energy_Services'], title='Energy services'),
    'rejected': ProcessGroup(['Rejected_Energy'], title='Rejected energy'),

    'direct_use': Waypoint(Partition.Simple('source', [
        # This is a hack to hide the labels of the partition, there should be a better way...
        (' '*i, [k]) for i, k in enumerate(sources)
    ])),
}

ordering = [
    [[], ['sources'], []],
    [['imports'], ['electricity', 'direct_use'], []],
    [[], ['uses'], []],
    [[], ['rejected', 'energy_services'], []]
]

bundles = [
    Bundle('sources', 'electricity'),
    Bundle('sources', 'uses', waypoints=['direct_use']),
    Bundle('electricity', 'uses'),
    Bundle('imports', 'uses'),
    Bundle('uses', 'energy_services'),
    Bundle('uses', 'rejected'),
    Bundle('electricity', 'rejected'),
]

In [ ]:
"""Define the colours to roughly imitate the original Sankey diagram"""
palette = {
    'Solar': 'gold',
    'Nuclear': 'red',
    'Hydro': 'blue',
    'Wind': 'purple',
    'Geothermal': 'brown',
    'Natural_Gas': 'steelblue',
    'Coal': 'black',
    'Biomass': 'lightgreen',
    'Petroleum': 'green',
    'Electricity': 'orange',
    'Rejected energy': 'lightgrey',
    'Energy services': 'dimgrey',
}

And here's the result!

In [ ]:
sdd = SankeyDefinition(nodes, bundles, ordering,
                       flow_partition=dataset.partition('type'))
weave(sdd, dataset, palette=palette) \
    .to_widget(width=700, height=450, margins=dict(left=100, right=120), debugging=True)

You can save a copy of the Sankey by adding `.auto_save_png('filename.png')` or `.auto_save_svg('filename.svg')` to the end of the `weave` call in the previous box.

## Task 2 - Create your own

Follow the steps below to create an equivalent Sankey for a different country.

  1. Find and download the IEA World Energy Balances Highlights spreadsheet, from the webpage: https://www.iea.org/data-and-statistics/data-product/world-energy-statistics-and-balances. Open it up and navigate to the `TimeSeries_1971-2022` sheet. Save this sheet as a .csv file which is much easier to read into most programmes than an Excel file. Then upload it to Colab using the `upload` button in the left panel or by simply dragging and dropping.

  2. In the next cell import the sheet to a pandas DataFrame. To find appropriate functions for the next steps either have a look at the [pandas documentation](https://pandas.pydata.org/docs/reference/index.html), or remember [your best friend](https://www.google.com/) when writing code.




In [ ]:
"""Read in an Excel file"""
import pandas as pd
fileName = 'WorldEnergyBalancesHighlights2023.csv'
data = pd.read_csv(fileName, skiprows=1)

3. Filter the DataFrame to contain only the desired country data.

In [ ]:
"""Get desired country"""
country = 'United Kingdom'
countryData = data.loc[data['Country']==country]

4. Filter the DataFrame to only contain 'Product', 'Flow' and value for the latest full year.

In [ ]:
"""Get values for latest year"""
lastYear = '2021'
filterData = countryData[['Product','Flow',lastYear]]

# Display data
display(filterData)

5. Filter out rows containing summaries (i.e. Total, Production), different units (GWh) or non-numeric values.

In [ ]:
"""Filter out Totals and bad characters"""
remove = '|'.join(['Production','Total','GWh'])
filterData = filterData[~filterData['Product'].str.contains(remove)]
filterData = filterData[~filterData['Flow'].str.contains(remove)]

filterData[lastYear] = filterData[lastYear].astype(int)
filterData = filterData[[type(i) is not str for i in filterData[lastYear]]]

6. Let's match the format in the files for the US example that you can find in the 'example_data' folder.

In [ ]:
"""Create dataset table"""
# Rename the columns to define source, target and value
filterData.rename(columns={'Product':'source', 'Flow':'target', lastYear:'value'}, inplace=True)

# Create type column
filterData['type'] = filterData['source']

# Get absolute values to display exports
filterData['value'] = abs(filterData['value'])

# Create groupings
groups = [['Electricity','Electricity'],['Oil products','Oil refineries']]
for g in groups:
  filterData['target'] = [g[0] if g[1] in i['target'] else i['target'] for i in filterData.iloc]

# Order data so that imports are considered a source and not a target
import numpy as np
orderData = filterData.copy()
importRows = np.where(['Imports' in i for i in filterData['target']])[0]
orderData['source'].iloc[importRows] = filterData['target'].iloc[importRows]
orderData['target'].iloc[importRows] = filterData['source'].iloc[importRows]

display(orderData)

6. Let's display all the individual sources and targets and attribute them to process groups for our Sankey diagram.

In [ ]:
"""Display individual sources and targets"""
display(orderData['source'].unique(), orderData['target'].unique())

In [ ]:
"""Attribute to process groups"""
sources = ['Coal, peat and oil shale', 'Crude, NGL and feedstocks', 'Natural gas', 'Nuclear', 'Renewables and waste','Heat']
uses = ['Industry (PJ)', 'Transport (PJ)', 'Residential (PJ)', 'Commercial and public services (PJ)',
       'Other final consumption (PJ)']
imports = ['Imports (PJ)']
exports = ['Exports (PJ)']
electricity = ['Electricity']
refining = ['Oil products']

7. Create process table as in us-energy-consumption-processes.csv

In [ ]:
"""Get all unique types of sources and targets listed in products and flows respectively"""
idColumn = np.concatenate((sources,uses))
typeColumn = ['source']*len(sources)+['use']*len(uses)
processes = pd.DataFrame(np.array([idColumn,typeColumn]).transpose(), columns=['id','type'])

We now have the same tables as used in the US example. So now copy the Sankey building boxes and see what you can do.

In [ ]:
"""Load the dataset"""
dataset = Dataset(orderData, dim_process=processes.set_index('id'))

6. Fetching the Sankey definition for the US energy consumption example, adapt this to fit with your new source and target values. Adapting the flows can be quite fiddly, we need to think effectively about the order of the flows.

In [ ]:
"""Define the Sankey diagram definition"""
nodes = {
    # Processes
    'sources': ProcessGroup('type == "source"', Partition.Simple('process', sources), title='Sources'),
    'imports': ProcessGroup(imports, title='Imports'),
    'exports': ProcessGroup(exports, title='Exports'),
    'electricity': ProcessGroup(electricity, title='Electricity\n Generation'),
    'uses': ProcessGroup('type == "use"', Partition.Simple('process', uses), title='Uses'),
    'refining': ProcessGroup(refining, title='Refining'),
    # Waypoints
    'direct_use': Waypoint(Partition.Simple('source', [(' '*i, [k]) for i, k in enumerate(sources)])),
    'direct_use_2': Waypoint(Partition.Simple('source', [(' '*i, [k]) for i, k in enumerate(sources)])),
    'direct_use_3': Waypoint(Partition.Simple('source', [(' '*i, [k]) for i, k in enumerate(sources)])),
    'electricity_use': Waypoint(Partition.Simple('source', ' ')),
    'refining_use':  Waypoint(Partition.Simple('source', ' ')),
    'refining_use_2': Waypoint(Partition.Simple('source', ' ')),
    'import_refining': Waypoint(Partition.Simple('source', ' ')),
    'import_electricity': Waypoint(Partition.Simple('source', ' ')),
    'import_electricity_2': Waypoint(Partition.Simple('source', (' ')))
}

ordering = [
    [[],['imports']],
    [['sources'],['import_refining','import_electricity']],
    [['direct_use','refining'], ['import_electricity_2']],
    [['direct_use_2','refining_use', 'electricity'],[]],
    [['direct_use_3','refining_use_2','electricity_use'], ['exports']],
    [['uses'], []],
]

bundles = [
    Bundle('imports','sources'),
    Bundle('sources', 'refining'),
    Bundle('sources', 'electricity', waypoints=['direct_use']),
    Bundle('sources', 'exports', waypoints=['direct_use','direct_use_2']),
    Bundle('sources', 'uses', waypoints=['direct_use','direct_use_2','direct_use_3']),
    Bundle('imports', 'refining', waypoints=['import_refining']),
    Bundle('refining', 'electricity'),
    Bundle('refining','exports', waypoints=['refining_use']),
    Bundle('refining', 'uses', waypoints=['refining_use','refining_use_2']),
    Bundle('imports','electricity', waypoints=['import_electricity','import_electricity_2']),
    Bundle('electricity', 'exports'),
    Bundle('electricity', 'uses', waypoints=['electricity_use'])
]

In [ ]:
"""Define the colours to roughly imitate the original Sankey diagram"""
palette = {
    'Coal, peat and oil shale': 'black',
    'Crude, NGL and feedstocks':'grey',
    'Oil products': 'purple',
    'Natural gas': 'steelblue',
    'Nuclear': 'red',
    'Renewables and waste':'green',
    'Electricity': 'orange',
    'Heat': 'red',
    'Fossil Fuels': 'darkgrey',
    'Renewable sources':'lightgreen'
}

In [ ]:
"""Draw out Sankey"""
sdd = SankeyDefinition(nodes, bundles, ordering,
                       flow_partition=dataset.partition('type'))
weave(sdd, dataset, palette=palette) \
    .to_widget(width=900, height=500, margins=dict(left=100, right=200)) \
.auto_save_svg(country+'Sankey.svg')

# Task 3 - Let's automate this procedure for any country with just one click.

Define a function that incorporates all of the previous steps while thinking about still being able to modify it from the outside. The `%%writefile` command writes this function to a Python file when the cell is run.

In [ ]:
%%writefile draw_sankey.py
import numpy as np
import pandas as pd
from floweaver import *

def draw_Country_Sankey(fileName:str, params_in:dict=False, diagramParams_in:dict=False):
    """This function creates a Sankey diagram for the data contained in the specified
    .xlsx file according to the specified parameters"""

    # Function definitions for each processing stage
    def get_country_data(fileName, countryName=False, headerRows=1):
        """Extract data for desired country from Excel sheet to pandas dataFrame"""
        data = pd.read_csv(fileName, skiprows=1)
        return data.loc[data['Country']==countryName] if countryName else data

    def filter_data(data, source, target, year='2021', skiprows=False):
        """Filter dataFrame according to year and removing rows with unused data"""
        if skiprows:
          remove = '|'.join(skiprows)
          data = data[~data[source].str.contains(remove)]
          data = data[~data[target].str.contains(remove)]

        data[year] = data[year].astype(int)
        data = data[[type(i) is not str for i in data[year]]]
        return data[[source,target,year]], year

    def format_data(data, colNames=False):
        """Format dataFrame to be in correct format for floweaver"""
        if colNames: data.rename(columns=colNames, inplace=True)
        data['type'], data['value'] = data['source'], abs(data['value'])
        return data

    def group_processes(data, groupings):
        """Group inputs and outputs that represent same process"""
        for g in groupings:
          data['target'] = [g[0] if g[1] in i['target'] else i['target'] for i in data.iloc]
        return data

    def reorder_data(data, reorders):
        """Order data so that imports are considered a source and not a target"""
        importRows = np.where([reorders in i for i in data['target']])[0]
        orderData = data.copy()
        orderData['source'].iloc[importRows] = data['target'].iloc[importRows]
        orderData['target'].iloc[importRows] = data['source'].iloc[importRows]
        return orderData

    def create_process_df(processes):
        """Get all unique types of sources and targets listed in products and flows respectively"""
        idColumn = np.concatenate((processes['sources'], processes['uses']))
        typeColumn = ['source']*len(processes['sources'])+['use']*len(processes['uses'])
        return pd.DataFrame(np.array([idColumn,typeColumn]).transpose(), columns=['id','type'])

    # Define default parameters
    params={
        'countryName': False,
        'source': 'Product',
        'target':'Flow',
        'year': False,
        'skiprows':['Production','Total','GWh'],
        'groupings':[['Electricity','Electricity'],['Oil products','Oil refineries']],
        'reordering':'Imports',
        'processes':{'sources':['Coal, peat and oil shale', 'Crude, NGL and feedstocks', 'Natural gas', 'Nuclear', 'Renewables and waste','Heat'],
                     'uses':['Industry (PJ)', 'Transport (PJ)', 'Residential (PJ)', 'Commercial and public services (PJ)', 'Other final consumption (PJ)'],
                     'imports':['Imports (PJ)'],
                     'exports':['Exports (PJ)'],
                     'electricity':['Electricity'],
                     'refining':['Oil products']}
    }

    diagramParams={
        'nodes': {
            'sources': ProcessGroup('type == "source"', Partition.Simple('process', params['processes']['sources']), title='Sources'),
            'imports': ProcessGroup(params['processes']['imports'], title='Imports'),
            'exports': ProcessGroup(params['processes']['exports'], title='Exports'),
            'electricity': ProcessGroup(params['processes']['electricity'], title='Electricity\n Generation'),
            'uses': ProcessGroup('type == "use"', Partition.Simple('process', params['processes']['uses']), title='Uses'),
            'refining': ProcessGroup(params['processes']['refining'], title='Refining'),
            'direct_use': Waypoint(Partition.Simple('source', [(' '*i, [k]) for i, k in enumerate(params['processes']['sources'])])),
            'direct_use_2': Waypoint(Partition.Simple('source', [(' '*i, [k]) for i, k in enumerate(params['processes']['sources'])])),
            'direct_use_3': Waypoint(Partition.Simple('source', [(' '*i, [k]) for i, k in enumerate(params['processes']['sources'])])),
            'electricity_use': Waypoint(Partition.Simple('source', ' ')),
            'refining_use':  Waypoint(Partition.Simple('source', ' ')),
            'refining_use_2': Waypoint(Partition.Simple('source', ' ')),
            'import_refining': Waypoint(Partition.Simple('source', ' ')),
            'import_electricity': Waypoint(Partition.Simple('source', ' ')),
            'import_electricity_2': Waypoint(Partition.Simple('source', (' ')))
            },

          'ordering': [
              [[],['imports']],
              [['sources'],['import_refining','import_electricity']],
              [['direct_use','refining'], ['import_electricity_2']],
              [['direct_use_2','refining_use', 'electricity'],[]],
              [['direct_use_3','refining_use_2','electricity_use'], ['exports']],
              [['uses'], []]
             ],

          'bundles': [
              Bundle('imports','sources'),
              Bundle('sources', 'refining'),
              Bundle('sources', 'electricity', waypoints=['direct_use']),
              Bundle('sources', 'exports', waypoints=['direct_use','direct_use_2']),
              Bundle('sources', 'uses', waypoints=['direct_use','direct_use_2','direct_use_3']),
              Bundle('imports', 'refining', waypoints=['import_refining']),
              Bundle('refining', 'electricity'),
              Bundle('refining','exports', waypoints=['refining_use']),
              Bundle('refining', 'uses', waypoints=['refining_use','refining_use_2']),
              Bundle('imports','electricity', waypoints=['import_electricity','import_electricity_2']),
              Bundle('electricity', 'exports'),
              Bundle('electricity', 'uses', waypoints=['electricity_use'])
              ],

          'palette': {
              'Coal, peat and oil shale': 'black',
              'Crude, NGL and feedstocks':'grey',
              'Oil products': 'purple',
              'Natural gas': 'steelblue',
              'Nuclear': 'red',
              'Renewables and waste':'green',
              'Electricity': 'orange',
              'Heat': 'red',
              'Fossil Fuels': 'darkgrey',
              'Renewable sources':'lightgreen'
              }
    }

    # Update default parameters if other parameters are passed to the function
    if params_in: params.update(params_in)
    if diagramParams_in: diagramParams.update(diagramParams)

    # Data processing
    countryData = get_country_data(fileName, params['countryName'])
    filterData, year = filter_data(countryData, params['source'],params['target'], params['year'],params['skiprows'])
    formattedData = format_data(filterData, colNames={params['source']:'source', params['target']:'target', year:'value'})
    groupData = group_processes(formattedData, params['groupings'])
    orderedData = reorder_data(groupData, params['reordering'])

    # Create processes
    processes = create_process_df(params['processes'])

    # Create Sankey Dataset
    dataset = Dataset(orderedData, dim_process=processes.set_index('id'))

    # Return sankey diagram
    sdd = SankeyDefinition(diagramParams['nodes'], diagramParams['bundles'], diagramParams['ordering'], flow_partition=dataset.partition('type'))
    return weave(sdd, dataset, palette=diagramParams['palette']).to_widget(width=900, height=500, margins=dict(left=100, right=200)) \
    .auto_save_svg(params['countryName']+'Sankey.svg')


Now import the function you've created from the written file. This could be done from any Jupyter notebook or python script.

In [ ]:
# Import function
import draw_sankey as ds

# Reloads function in case you've already imported it to this notebook
from importlib import reload
reload(ds)

In [ ]:
# Define variables
fileName = 'WorldEnergyBalancesHighlights2023.csv'
country = 'France'
year = '2021'

# Call function
diagram = ds.draw_Country_Sankey(fileName, {'year':year, 'countryName':country})

# Display Sankey
display(diagram)

If this has worked then have a play around and make sure it works for any country by simply modifying the `country` variable.

The solution example is written according to the principles of functional programming, but this could also be written according to object-oriented programming by defining the sankey diagram as a `Class`. If you are a Python expert already and feeling bored why not have a go at re-writing the draw_sankey.py file as a Class definition. Alternatively you could solidify the function you've created with error messages to catch bad inputs or even create a widget which allows you to pick the country you're after off a list rather than typing in the variable.